# Assigment 3


## Upload your code on Learn dropbox and submit pdfs of the code and answers to the mathematical questions on Crowdmark.

## -----------------------------------------------------------------------------------------------------------

## Load modules

In [ ]:
# !pip install numpy, scipy, scikit-image, skimage, matplotlib

import matplotlib.pyplot as plt

from skimage.color import rgb2gray
from skimage import data
from skimage.transform import resize

# Numpy is useful for handling arrays and matrices.
import numpy as np
import scipy
import time

## Load image

In [ ]:
img = data.astronaut()
img = rgb2gray(img)*255 # convert to gray and change scale from (0,1) to (0,255).

n = img.shape[0]
m = n

plt.figure(1, figsize=(10, 10))
plt.imshow(img, cmap='gray', vmin=0, vmax=255)
plt.show()

## Compute the differences operators here. Use your code from Assignment 2.

In [ ]:
# You will need these three methods to construct sparse differences operators.
# If you do not use sparse operators you might have scalability problems.
from scipy.sparse import diags
from scipy.sparse import kron
from scipy.sparse import identity
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
from scipy import real
# Use your code from Assignment 2. 
# Make sure that you compute the right D_h and D_v matrices.

J = diags(diagonals=[
    [-1.0] * n,
    [1.0] * (n-1)
], offsets=[0, 1])

I = identity(n=n)

# Forward Horizontal Difference
D_h = kron(J, I)

# Forward Vertical Difference
D_v = kron(I, J)

D = D_h + 1j * D_v

D_s = real((D.H).dot(D))
I_s = identity(n*m)

# Matrix norm with tolerance parameter to control speed vs. precision tradeoff
def matrix_p2_norm(A, tol=0.001):
    eig_val = eigsh((A.H).dot(A), return_eigenvectors=False, tol=tol, k=1)
    return np.sqrt(eig_val).sum()

# Psuedo-Huber function for vector v, to replace L1 norm
def psf(v, mu):
    return np.sqrt(mu ** 2.0  + np.power(np.abs(v), 2.0)) - mu

# The total-variation denoising objective function
def f_tv(lamb, mu, x, z_n):
    return real(lamb * psf(v=D.dot(x), mu=mu).sum() + 0.5 * np.linalg.norm(x - z_n) ** 2.0)

# The total-variation denoising objective function gradient
def grad_f_tv2(lamb, mu, x, z_n):
    x = csr_matrix(x)
    
    dhx = D_h.dot(x)
    dvx = D_v.dot(x)
    
    numer = D_h.multiply(dhx) + D_v.multiply(dvx)
    
    d_x_abs = dhx.power(2.0) + dvx.power(2.0)
    denom = np.sqrt(mu ** 2.0 + d_x_abs.toarray())
    
    frac = numer.multiply(1.0 / denom)
    return lamb * frac.sum(axis=0).T  + x - z_n

## Add noise to the image

In [ ]:
x = np.arange(-0.01, 0.01, step=0.002)
y_abs = np.abs(x)
y_mu1 = psf(x, mu=0.001)
y_mu2 = psf(x, mu=0.01)
y_mu3 = psf(x, mu=0.1)

fig = plt.figure(figsize=(8, 6))
plt.plot(x, y_abs, 'r--', x, y_mu1, 'g--', x, y_mu2, 'b--', x, y_mu3, 'm--')
plt.grid(linestyle='dashed')
plt.show()

In [ ]:
mean_ = 0
standard_deviation = 30
dimensions = (n,n)

noise = np.random.normal(mean_,standard_deviation,dimensions)

noisy_image = img + noise
noisy_image_vec = csr_matrix(noisy_image.T.ravel()).T
image_vec = csr_matrix(img.T.ravel()).T

plt.figure(1, figsize=(10, 10))
plt.imshow(noisy_image, cmap='gray', vmin=0, vmax=255)
plt.show()

## Question 1 (8 marks): implement gradient descent with Armijo line-search for the Total-Variation denoising problem. Use the pseudo-Huber function to smooth the problem. 

In [ ]:
def line_search_arm(lambda_, mu, x, f_x, grad_f_x, norm_grad_f_x, gamma):
    # lambda_: the regularization parameter
    # x: the current estimate of t=he variable
    # f_x: the value of the objective function at x
    # grad_f_x: the gradient of the objective function at x
    # norm_grad_f_x: the norm of grad_f_x
    # gamma: parameter of Armijo line-search as was defined in the lectures.

    alpha = 1.0
    loss = gamma * (norm_grad_f_x ** 2.0)
    while f_tv(lamb=lambda_, mu=mu, x=x-alpha*grad_f_x, z_n=noisy_image_vec) > f_x - alpha * loss:
        alpha /= 2.0
    return alpha
    
def gradient_descent_arm(x0, epsilon, lambda_, mu, max_iterations, gamma):
    # x0: is the initial guess for the x variables
    # epsilon: is the termination tolerance parameter
    # lambda_: is the regularization parameter of the denoising problem.
    # max_iterations: is the maximum number of iterations that you allow the algorithm to run.
    # gamma: parameter of Armijo line-search as was defined in the lectures.
    
    x_updated = x0.copy().toarray()
    f_vals = []
    norm_vals = []
    t1 = time.time()
    for i in range(1, max_iterations+1):
        current_grad = grad_f_tv2(lamb=lambda_, mu=mu, x=x_updated, z_n=noisy_image_vec)
        tx = time.time()
        current_grad_norm = np.linalg.norm(current_grad)
        if current_grad_norm <= epsilon:
            break
        norm_vals.append(current_grad_norm)
        f_vals.append(f_tv(lamb=lambda_, mu=mu, x=x_updated, z_n=noisy_image_vec))
        alpha = line_search_arm(lambda_=lambda_, mu=mu, x=x_updated, f_x=f_vals[-1], grad_f_x=current_grad, norm_grad_f_x=current_grad_norm, gamma=gamma)
        x_updated = x_updated - alpha * current_grad
        f_diff = (f_vals[-1] - f_vals[-2]) if len(f_vals) > 1 else None
        grad_diff = (norm_vals[-1] - norm_vals[-2]) if len(norm_vals) > 1 else None
        print(f"Step = {i}: alpha = {alpha}, Function = {f_vals[-1]}, Function Diff. =  {f_diff}, Grad. Norm = {norm_vals[-1]}, Grad. Norm. Diff. = {grad_diff}")
    t2 = time.time()
    print(f"Iterations (Total) time = {t2-t1}")
    return x_updated, np.array(f_vals)

## Call Gradient Descent with Armijo line-search to denoise the image. Parameter tunning is not given for this assignment. You will have to tune all parameters yourself. Regarding the quality of the output image, pick the $\lambda$ parameter that makes the error $$\frac{1}{n^2}\|z_{output} - z_{clean}\|_2$$ as small as possible, where $z_{output}$ is the output of the algorithm. Find $\lambda$ by trial and error. Note that the smoothing parameter $\mu$ affects the quality of the output as well. Pick $\mu$ small enough such that the above error does not improve much for smaller values of $\mu$. I will measure the running time only for your chosen parameters $\lambda$ and $\mu$, therefore, make sure to seperate any code that does trial and error and the code that reports the result for the chosen parameters.

In [ ]:
# Initialize parameters of gradient descent
lambda_ = 45
mu = 0.1
epsilon = 1.0e-2
gamma = 0.3
max_iterations = 200

# Set x0 equal to the vectorized noisy image.
# Write your code here.
optimized_gd_arm, f_vals_arm = gradient_descent_arm(x0=noisy_image_vec, 
                                                    lambda_=lambda_,
                                                    mu=mu,
                                                    epsilon=epsilon,
                                                    gamma=gamma,
                                                    max_iterations=max_iterations)

In [ ]:
# mu = 0.01
    # 10: 8515.536787930056
    # 25: 5403.552568529774
    # 30: 5218.644577836724
    # 40: 5092.992288363075
    # 45: 5080.710291882298
    # 50: 5104.336472912059
    # 60: 5201.170909978507
    # 70: 5349.886968970723
# mu = 0.001
    # 45: 5080.537317632449
vector_norm(optimized_gd_arm - image_vec)

In [ ]:
print(np.linalg.norm(optimized_gd_arm - image_vec))
denoised_image_gd_arm = optimized_gd_arm.reshape((m, n), order='F')
plt.figure(1, figsize=(10, 10))
plt.imshow(denoised_image_gd_arm, cmap='gray', vmin=0, vmax=255)
plt.show()

## Question 2 (5 marks): implement gradient descent with simple line-search for the Total-Variation denoising problem. Use the pseudo-Huber function to smooth the problem.

In [ ]:
def line_search_ls(lambda_, mu, x, f_x, grad_f_x):
    # lambda_: the regularization parameter
    # x: the current estimate of t=he variable
    # f_x: the value of the objective function at x
    # grad_f_x: the gradient of the objective function at x
    
    alpha = 1.0
    while f_tv(lamb=lambda_, mu=mu, x=x-alpha*grad_f_x, z_n=noisy_image_vec) >= f_x:
        alpha /= 2.0
    return alpha

# Write gradient descent + line-search here.
def gradient_descent_ls(x0, epsilon, lambda_, mu, max_iterations):
    # x0: is the initial guess for the x variables
    # epsilon: is the termination tolerance parameter
    # lambda_: is the regularization parameter of the denoising problem.
    # max_iterations: is the maximum number of iterations that you allow the algorithm to run.

    x_updated = x0.copy().toarray()
    f_vals = []
    norm_vals = []
    t1 = time.time()
    for i in range(1, max_iterations+1):
        current_grad = grad_f_tv2(lamb=lambda_, mu=mu, x=x_updated, z_n=noisy_image_vec)
        current_grad_norm = np.linalg.norm(current_grad)
        if current_grad_norm <= epsilon:
            break
        norm_vals.append(current_grad_norm)
        f_vals.append(f_tv(lamb=lambda_, mu=mu, x=x_updated, z_n=noisy_image_vec))
        alpha = line_search_ls(lambda_=lambda_, mu=mu, x=x_updated, f_x=f_vals[-1], grad_f_x=current_grad)
        x_updated = x_updated - alpha * current_grad
        f_diff = (f_vals[-1] - f_vals[-2]) if len(f_vals) > 1 else None
        grad_diff = (norm_vals[-1] - norm_vals[-2]) if len(norm_vals) > 1 else None
        print(f"Step = {i}: alpha = {alpha}, Function = {f_vals[-1]}, Function Diff. =  {f_diff}, Grad. Norm = {norm_vals[-1]}, Grad. Norm. Diff. = {grad_diff}")
    t2 = time.time()
    print(f"Iterations (Total) time = {t2-t1}")
    return x_updated, np.array(f_vals)

## Call gradient descent with simple line-search to denoise the image. Use the same $\lambda$ and $\mu$ that you used in Q1. 

In [ ]:
# Initialize parameters of gradient descent
lambda_ = 45
mu = 0.1
epsilon = 1.0e-2
max_iterations = 200

# Set x0 equal to the vectorized noisy image.
# Write your code here.
optimized_gd_ls, f_vals_ls = gradient_descent_ls(x0=noisy_image_vec,
                                                 lambda_=lambda_, 
                                                 mu=mu,
                                                 epsilon=epsilon, 
                                                 max_iterations=max_iterations)

In [ ]:
print(np.linalg.norm(optimized_gd_ls - image_vec))
denoised_image_gd_ls = optimized_gd_ls.reshape((m, n), order='F')
plt.figure(1, figsize=(10, 10))
plt.imshow(denoised_image_gd_ls, cmap='gray', vmin=0, vmax=255)
plt.show()

## Question 3 (6 marks): Compute a Lipschitz constant for the smoothed Total-Variation problem. Note the Lipschitz constant is not unique. However, the minimum Lipschitz constant will give you better performance in algorithms compared to larger Lipschitz constants. This means that whatever you compute here will affect the running time of your algorithm in Q4.

In [ ]:
def compute_lipschitz_tv(lamb, mu):
    A = (lamb / mu) * D_s + I_s
    g = matrix_p2_norm(A, tol=0.001)
    return g.sum()

In [ ]:
lambda_ = 45
mu = 0.1
L = compute_lipschitz_tv(lamb=lambda_, mu=mu)

In [ ]:
L

## Question 4 (8 marks): implement accelerated gradient for the Total-Variation denoising problem. Use the pseudo-Huber function to smooth the problem. Use the Lipschitz constant that you obtained in Q3. Do not include computation of the Lipschitz constant in this question. You can do it in Q3 and the time for computing the Lipschitz constant will not be taken into account.

In [ ]:
def accelerated_gd_lip(x0, epsilon, lambda_, mu, max_iterations):
    z = x0.copy().toarray()
    x_updated = x0.copy().toarray()
    f_vals = []
    norm_vals = []
    t1 = time.time()
    alpha = 1.0 / L
    lamb = 1.0
    for i in range(1, max_iterations+1):
        gamma = 2.0 / i if i > 4 else 0
        y = (1.0 - gamma) * x_updated + gamma * z
        current_grad = grad_f_tv2(lamb=lambda_, mu=mu, x=y, z_n=noisy_image_vec)
        current_grad_norm = np.linalg.norm(current_grad)
        if current_grad_norm <= epsilon:
            break
        norm_vals.append(current_grad_norm)
        
        z = z - alpha * (gamma / lamb) * current_grad
        f_vals.append(f_tv(lamb=lambda_, mu=mu, x=x_updated, z_n=noisy_image_vec))
        x_updated = y - alpha * current_grad
        lamb = (1.0 - gamma) * lamb
        f_diff = (f_vals[-1] - f_vals[-2]) if len(f_vals) > 1 else None
        grad_diff = (norm_vals[-1] - norm_vals[-2]) if len(norm_vals) > 1 else None
        print(f"Step = {i}: alpha = {alpha}, gamma = {gamma}, lambda = {lamb}, Function = {f_vals[-1]}, Function Diff. =  {f_diff}, Grad. Norm = {norm_vals[-1]}, Grad. Norm. Diff. = {grad_diff}")
    t2 = time.time()
    print(f"Iterations (Total) time = {t2-t1}")
    return x_updated, np.array(f_vals)

## Call accelerated gradient to denoise the image. Use the same $\lambda$ and $\mu$ that you used in Q1.

In [ ]:
# Initialize parameters of gradient descent
lambda_ = 45
mu = 0.1
epsilon = 1.0e-2
max_iterations = 200

# Set x0 equal to the vectorized noisy image.
# Write your code here.
optimized_gd_acc_lip, f_vals_acc_lip = accelerated_gd_lip(x0=noisy_image_vec,
                                                          lambda_=lambda_, 
                                                          mu=mu,
                                                          epsilon=epsilon, 
                                                          max_iterations=max_iterations)

In [ ]:
print(np.linalg.norm(optimized_gd_acc_lip - image_vec))
denoised_image_gd_acc_lip = optimized_gd_acc_lip.reshape((m, n), order='F')
plt.figure(1, figsize=(10, 10))
plt.imshow(denoised_image_gd_acc_lip, cmap='gray', vmin=0, vmax=255)
plt.show()

## Question 5 (10 marks): in Q4 you were asked to implement accelerated gradient by using constant step-sizes $1/L.$ However, computing the Lipschitz constant might take a lot of time and it often results in slow convergence because the step-sizes are too small. Below I give you a practical accelerated method that does not require knowing the Lipschitz constant. The step-sizes $$\alpha_k$$ in this algorithm can be computed using Armijo line-search. Implement this algorithm for the Total-Variation denoising problem. Use the pseudo-Huber function to smooth the problem. 

Step 1) Choose an $x_0$ and set $y_1=x_0$, $t_1=1$.

Step 2) Repeat the following steps until $\|\nabla f(x_k)\|_2\le \epsilon$

Step 3) Compute $\alpha_k$ using Armijo line-search. Armijo line-search should be measured at $y_k-\alpha_k \nabla f(y_k)$ (as the next point) and $y_k$ (as the current point).

Step 4) Set $$x_{k}=y_k - \alpha_k \nabla f(y_k)$$

Step 5) Set $$t_{k+1}=\frac{1 + \sqrt{1+4t_k^2}}{2}$$

Step 6) Set $$y_{k+1}=x_k + \frac{t_k-1}{t_{k+1}}(x_k-x_{k-1})$$

Reference: this algorithm is given in "A Fast Iterative Shrinkage-Thresholding Algorithm for Linear Inverse Problems" by A. Beck and M. Teboulle.
===================== 

In [ ]:
def line_search_arm_prac(lambda_, mu, x, f_x, grad_f_x, norm_grad_f_x, gamma):
    # lambda_: the regularization parameter
    # x: the current estimate of t=he variable
    # f_x: the value of the objective function at x
    # grad_f_x: the gradient of the objective function at x
    # norm_grad_f_x: the norm of grad_f_x
    # gamma: parameter of Armijo line-search as was defined in the lectures.

    # Write your code here.
    alpha = 1.0
    loss = gamma * (norm_grad_f_x ** 2.0)
    while f_tv(lamb=lambda_, mu=mu, x=x-alpha*grad_f_x, z_n=noisy_image_vec) > f_x - alpha * loss:
        alpha /= 2.0
    return alpha

def accelerated_gd_practical(x0, epsilon, lambda_, mu, max_iterations, gamma):
    y = x0.copy().toarray()
    t = 1.0
    x_updated = x0.copy().toarray()
    f_vals = []
    norm_vals = []
    t1 = time.time()
    for i in range(1, max_iterations+1):
        current_grad = grad_f_tv2(lamb=lambda_, mu=mu, x=y, z_n=noisy_image_vec)
        current_grad_norm = np.linalg.norm(current_grad)
        if current_grad_norm <= epsilon:
            break
        norm_vals.append(current_grad_norm)
        fy = f_tv(lamb=lambda_, mu=mu, x=y, z_n=noisy_image_vec)
        alpha = line_search_arm_prac(lambda_=lambda_, mu=mu, x=y, f_x=fy, grad_f_x=current_grad, norm_grad_f_x=current_grad_norm, gamma=gamma)
        f_vals.append(f_tv(lamb=lambda_, mu=mu, x=x_updated, z_n=noisy_image_vec))
        
        x_updated_new = y - alpha * current_grad
        t_new = (1.0 + np.sqrt(1 + 4 * t ** 2)) / 2.0
        y = x_updated_new + ((t - 1.0) / t_new) * (x_updated_new - x_updated)
        
        t = t_new
        x_updated = x_updated_new
        
        f_diff = (f_vals[-1] - f_vals[-2]) if len(f_vals) > 1 else None
        grad_diff = (norm_vals[-1] - norm_vals[-2]) if len(norm_vals) > 1 else None
        print(f"Step = {i}: alpha = {alpha}, t = {t}, Function = {f_vals[-1]}, Function Diff. =  {f_diff}, Grad. Norm = {norm_vals[-1]}, Grad.Norm. Diff. = {grad_diff}")
    t2 = time.time()
    print(f"Iterations (Total) time = {t2-t1}")
    return x_updated, np.array(f_vals)

## Call the practical accelerated gradient to denoise the image. Use the same $\lambda$ and $\mu$ that you used in Q1.

In [ ]:
# Initialize parameters of gradient descent
lambda_ = 45
mu = 0.1
epsilon = 1.0e-2
gamma = 0.3
max_iterations = 200

# Set x0 equal to the vectorized noisy image.
# Write your code here.
optimized_gd_acc_prac, f_vals_acc_prac = accelerated_gd_practical(x0=noisy_image_vec,
                                                                  lambda_=lambda_, 
                                                                  mu=mu,
                                                                  epsilon=epsilon, 
                                                                  max_iterations=max_iterations,
                                                                  gamma=gamma)

In [ ]:
print(np.linalg.norm(optimized_gd_acc_prac - image_vec))
denoised_image_gd_acc_prac = optimized_gd_acc_prac.reshape((m, n), order='F')
plt.figure(1, figsize=(10, 10))
plt.imshow(denoised_image_gd_acc_prac, cmap='gray', vmin=0, vmax=255)
plt.show()

## Question 6 (5 marks): Compare all the methods that you implemented above. Make a plot where in the y-axis is the value of the smoothed objective function and in the x-axis the number of iterations. Compare the methods using the same $\lambda$ and $\mu$ that you used in Q1.

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(1, 1, 1)
ax.plot(f_vals_acc_prac, label=("Practical Accelerated"), linewidth=3.0, color ="black")
ax.plot(f_vals_acc_lip, label=("Simplified Accelerated + Lip"), linewidth=3.0, color ="green")
ax.plot(f_vals_ls, label=("Simple Line Search"), linewidth=3.0, color ="magenta")
ax.plot(f_vals_arm, label=("Armijo Line Search"), linewidth=3.0, color ="blue")
# ax.set_yscale('log')
ax.legend(prop={'size': 15},loc="upper right")
plt.xlabel("iteration $k$", fontsize=25)
plt.ylabel("Smoothed Objective Function", fontsize=25)
ax.grid(linestyle='dashed')
plt.savefig('bar.pdf')

## Question 7 (8 marks): Illustrate the trade-off between the number of iterations and the smoothing parameter $\mu$ for gradient descent with Armijo line-search and accelerated gradient with Armijo line-search. Do this by plotting the number of iterations (y-axis) vs magnitude of parameter $\mu$ (x-axis in ascending order). Start from a small $\mu$ and increase it gradually. Plot the result for both methods in the same plot. Use appropriate legends for the plot.

In [ ]:
# This cell takes around 15 minutes, the output is provided separatly for reference

mus = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 2.0, 5.0, 10.0]

lambda_ = 45
epsilon = 1.0e-2
gamma = 0.3
max_iterations = 200

fvals_arm = []
fvals_prac = []
for mu_val in mus:
    optimized_gd_arm_loop, f_vals_arm_loop = gradient_descent_arm(x0=noisy_image_vec, 
                                                                  lambda_=lambda_,
                                                                  mu=mu_val,
                                                                  epsilon=epsilon,
                                                                  gamma=gamma,
                                                                  max_iterations=max_iterations)
    
    optimized_gd_acc_prac_loop, f_vals_acc_prac_loop = accelerated_gd_practical(x0=noisy_image_vec,
                                                                                lambda_=lambda_, 
                                                                                mu=mu_val,
                                                                                epsilon=epsilon, 
                                                                                max_iterations=max_iterations,
                                                                                gamma=gamma)
    fvals_arm.append(f_vals_arm_loop)
    fvals_prac.append(f_vals_acc_prac_loop)

In [ ]:
f_cutoff = 1.9e8
iter_arm = [np.where(fval <= f_cutoff)  for fval in fvals_arm]
iter_prac = [np.where(fval <= f_cutoff)  for fval in fvals_prac]

iter_arm = [max_iterations if len(i[0]) == 0 else i[0][0] for i in iter_arm]
iter_prac = [max_iterations if len(i[0]) == 0 else i[0][0] for i in iter_prac]

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(1, 1, 1)
ax.plot(mus, iter_arm, label=("Armijo Line Search"), linewidth=3.0, color ="black", linestyle='--')
ax.plot(mus, iter_prac, label=("Practical Accelerated"), linewidth=3.0, color ="green", linestyle='--')
# ax.set_yscale('log')
ax.set_xscale('log')
ax.legend(prop={'size': 15},loc="upper right")
plt.xlabel("$\mu$ (LOG)", fontsize=25)
plt.ylabel(f"Iterations to objective = {f_cutoff:.2E}", fontsize=25)
ax.grid(linestyle='dashed')
# plt.show()
plt.savefig('foo.pdf')

## Mathematical Questions

## Question 8 (5 marks): Derive a smooth approximation (Huber function) of the L1-norm using the convex conjugate of the L1-norm and the distance function $$d(y)=\frac{1}{2}\|y\|_2^2.$$

Solved

## Question 9 (7 marks): Derive the pseudo-Huber function using the convex conjugate of the L1-norm.  To derive the pseudo-Huber use the distance function $$d(y)=\sum_{i=1}^n 1 - \sqrt{1-y_i^2}$$ with domain $$|y_i|\le 1 \ \ \forall i.$$

Solved

## Question 10 (3 marks): Show that the pseudo-Huber function is convex and not strongly-convex.

Solved

## Question 11 (7 marks): If the convex conjugate of $f$ is strongly-convex with compact convex domain then $$\max_{y \in \mbox{dom} \ f^*} x^Ty - f^*(y)$$ has a unique maximizer. Assume that $f$ is convex and show that  $$\nabla f(x) =\mbox{argmax}_{y \in \mbox{dom} \ f^*} x^Ty - f^*(y).$$ Hint: one approach to solve this is to lower and upper bound $f$ and then use the definition of directional derivative.

Solved

## Question 12 (7 marks): Prove that if the convex conjugate of $f$ is $\delta$-strongly-convex, then $\nabla f(x)$ is Lipschitz continuous with Lipschitz constant $1/\delta.$

Solved

## Question 13 (8 marks):  Assume that the domain of the convex conjugate satisfies $$\mbox{dom} \ f^* \subseteq \mathbb{R}^n.$$ and it is closed and bounded. Also, assume that $f$ is convex. You are given a continuous distance function $d(y)$ where its domain satisfies $$\mbox{dom} \ f^* \subseteq \mbox{dom} \ d .$$ Prove that $$f(x) - \mu D \le f_\mu(x) \le f(x),$$ where $D$ has to be bounded and $f_\mu(x)$ is the smooth approximation of $f$. See slides 54-57 in Lecture07and08 on piazza. Hint: you will need the Weierstrass extreme value theorem to show that $D$ is bounded.

## Question 14 (8 marks): We proved that for any convex function with Lipschitz continuous gradient there exists and estimate sequence. Show that for an estimate sequence we have that $$f(x_k) - f^* = \mathcal{O}\left(\frac{1}{k^2}\right)$$

## Question 15 (5 marks): Obtain the convergence rate and iteration complexity for the accelerated method for strongly-convex functions with Lipschitz continuous gradient.